In [1]:
import logging
import sys

logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)

In [2]:
from bedarev_semantic.base import *


information = [
    #
    Synonyms(
        'ДолжностноеЛицо',
        {'рабочий', 'сотрудник', 'государственный служащий'}
    ),
    Synonyms(
        'РФ',
        {'рф', 'россия', 'российский федерация'}
    ),
    Synonyms(
        'Семья',
        {'семья', 'ячейка общество'}
    ),
    Synonyms(
        'ЦБ',
        {'цб', 'центральный банк', 'отечественный цб'}
    ),
    Synonyms(
        'ВВП',
        {'ввп', 'валовый продукт'}
    ),
    Synonyms(
        'ОбразованныйЧеловек',
        {'учёный', 'эксперт', 'академик'}
    ),
    GeneralPrivate(
        'Государство',
        {
            Synonyms('РФ'),
            'cccp',
            'китай',
            'сша'
        },
        values={'государство', 'страна', 'федерация'}
    ),
    GeneralPrivate(
        'ДенежныеСредства',
        {
            'рубль',
            'доллар',
        },
        {'деньга', 'средства', 'криптовалюта', 'валюта'}
    ),
    #
    GeneralPrivate(
        'Числительные',
        {
            'нуль',
            'три',
            'два',
            'миллион',
            'миллиардмиллиард',
        },
        {'число', 'количество'}
    ),
    # транзакции -> приход и расход
    GeneralPrivate(
        'Субъект',
        {
            'округ',
            'страна',
        },
        {'субъект', }
    ),
    PartWhole(
        'ПланетнаяСистема',
        {
            GeneralPrivate('Звезда', {'солнце'}, {'звезда'}),
            GeneralPrivate('Спутник', {'луна'}, {'спутник'}),
            GeneralPrivate('Планета', {'земля'}, {'планета'})
        },
        {'планетный система', }
    ),
    GeneralPrivate(
        'НаучнаяОбласть',
        {
            'математика',
            'физика',
            'информационные технологии',
        },
        {'наука'}
    ),
    Synonyms(
        'ПротопланетныйДиск',
        {'протопланетный диск', 'протопланетный кольцо'}
    ),
    Synonyms(
        'Затрата',
        {'затрата', 'издержка'}
    ),
    Synonyms(
        'Финансирование',
        {'кредит', 'кредитование',
         'вклад', 'финансирование', 'инвестиция'}
    ),
    Synonyms(
        'Налогообложение',
        {'налог', 'налоговый нагрузка'}
    ),
    Synonyms(
        'Регулирование',
        {'регулирование', 'управление'}
    ),
    Synonyms(
        'Банк',
        {'банк', 'банковский система'}
    ),
    PartWhole(
        'Правительство',
        {
            'мгд',
            'институт'
        },
        {'правительство', 'власть'}
    ),
    GeneralPrivate(
        'ПолезноеИскопаемое',
        {
            'газ',
            'нефть',
        },
        {'полезный ископаемое', 'ресурс'}
    ),
    PartWhole(
        'ВременнойОтрезок',
        {
            'x годах',
            Synonyms(
                'НастоящееВремя',
                {'последний время', 'сегодня', 'сейчас', 'настоящее время'}
            ),
        },
    ),
    Synonyms(
        'Аглоритм',
        {'алгоритм', 'технология', 'схема', 'подход'}
    ),
]

semantic_vocabulary = SemanticVocabulary(information)

INFO:pymorphy2.opencorpora_dict.wrapper:format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [3]:
import json

from bedarev_semantic.weighing import WeighingAlgorithm, TopDownAlgorithm
from bedarev_semantic.search_engine import SearchEngine

with open("./data/clear_text.txt", 'r', encoding='utf8') as fp:
    text_lines = fp.read().split('\n')
# with open('./data/all_forms.json', 'r', encoding='utf8') as fp:
#     all_forms = set(json.load(fp))

search_engine = SearchEngine(corpus=text_lines, all_forms=None, semantic_vocabulary=semantic_vocabulary)
len(search_engine.all_forms)

INFO:pymorphy2.opencorpora_dict.wrapper:Loading dictionaries from D:\ProgramData\Anaconda3\lib\site-packages\pymorphy2_dicts\data
INFO:pymorphy2.opencorpora_dict.wrapper:format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


766091

In [4]:
from typing import Set

def gener(phrase: str) -> Set[GeneralPrivate]:
    result = set()
    
    morhp_obj = semantic_vocabulary.search_morph_object(phrase)
    if not morhp_obj:
        return result
    for parent in morhp_obj.parents:
        if isinstance(parent, GeneralPrivate):
            result.add(parent)
    return result

In [5]:
print(gener('физика'))
search_engine.map(gener('физика'))[:2]

{<GeneralPrivate: "НаучнаяОбласть">
}
INFO:bedarev_semantic:weights: {(1, 'наука')}
INFO:bedarev_semantic:vector_request:   (0, 348014)	1


[(4109,
  6,
  {'наука': [95, 126, 144, 165, 185, 226]},
  'Хотя, разумеется, роль ретроспективного исследования научных достижений в разработке философии науки очень велика. «Философия науки без истории науки пуста; история науки без философии науки слепа», – пишет выдающийся философ науки И. Лакатос [14]. Создатель теории научных революций Т. Кун обращался к достижениям Птолемея, Коперника, Ньютона, Лавуазье, Дальтона. Концепция исследовательских программ И. Лакатоса строилась на основе анализа работ Бора и Эйнштейна, опыта становления квантовой механики и теории относительности.'),
 (3445,
  5,
  {'наука': [11, 29, 50, 70, 111]},
  '«Философия науки без истории науки пуста; история науки без философии науки слепа», – пишет выдающийся философ науки И. Лакатос. Создатель теории научных революций Т. Кун обращался к достижениям Птолемея, Коперника, Ньютона, Лавуазье, Дальтона [15]. Концепция исследовательских программ И. Лакатоса строилась на основе анализа работ Бора и Эйнштейна, опыта

In [6]:
print(gener('нефть'))
search_engine.map(gener('нефть'))[:2]

{<GeneralPrivate: "ПолезноеИскопаемое">
}
INFO:bedarev_semantic:weights: {(1, 'полезный_ископаемое'), (1, 'ресурс')}
INFO:bedarev_semantic:vector_request:   (0, 464076)	1
  (0, 558555)	1


[(3855,
  4,
  {'ресурс': [128, 190, 298, 518]},
  'При этом экономисты отмечают очевидное «несходство социальной политики, которую узкий круг обладателей имущественных и властных ресурсов осуществляет в отношении большинства, лишенного этих ресурсов, с той, которая строится на основе репрезентации интересов и предпочтений большинства, обладающего ресурсами» (Якобсон, 2016, с. 296–297)с. 296–297). На возможность более низкой оценки полезности общественных средств для людей, принимающих решения о направлениях государственных расходов, по отношению к их собственным ресурсам указывают многие факты14.'),
 (4146,
  4,
  {'ресурс': [40, 332, 497, 514]},
  'Подведомственные Министерству природных ресурсов и экологии Российской Федерации федеральные государственные бюджетные учреждения осуществляют организацию функционирования и охрану государственных природных заповедников, национальных парков и федеральных заказников [4].Кроме того, сегодня немаловажную роль в управлении природными ресурсами

In [7]:
[value.normal_form for value in list(gener('нефть'))[0].values()]

['полезный_ископаемое', 'ресурс']

In [8]:
print(gener('физика') | gener('нефть'))
search_engine.map(gener('физика') | gener('нефть'))[:1]

{<GeneralPrivate: "НаучнаяОбласть">
, <GeneralPrivate: "ПолезноеИскопаемое">
}
INFO:bedarev_semantic:weights: {(1, 'полезный_ископаемое'), (1, 'наука'), (1, 'ресурс')}
INFO:bedarev_semantic:vector_request:   (0, 348014)	1
  (0, 464076)	1
  (0, 558555)	1


[(4109,
  6,
  {'наука': [95, 126, 144, 165, 185, 226]},
  'Хотя, разумеется, роль ретроспективного исследования научных достижений в разработке философии науки очень велика. «Философия науки без истории науки пуста; история науки без философии науки слепа», – пишет выдающийся философ науки И. Лакатос [14]. Создатель теории научных революций Т. Кун обращался к достижениям Птолемея, Коперника, Ньютона, Лавуазье, Дальтона. Концепция исследовательских программ И. Лакатоса строилась на основе анализа работ Бора и Эйнштейна, опыта становления квантовой механики и теории относительности.')]

In [9]:
# search_engine.normalize_lines_with_grams[4720]